pip install openpyxl   
pip install xlrd

In [2]:
import numpy as np
import pandas as pd
import requests,json,os
from urllib.parse import quote 

In [3]:
df = pd.read_excel('./전체_도시철도역사정보_20230317.xlsx',engine='openpyxl')
df.head()

,역번호,역사명,노선번호,노선명,영문역사명,한자역사명,환승역구분,환승노선번호,환승노선명,역위도,역경도,운영기관명,역사도로명주소,역사전화번호,데이터기준일자
0,0101,판암,S3001,대전 도시철도 1호선,Panam,板岩,일반역,NaN,NaN,36.316896,127.458264,대전교통공사,대전광역시 동구 옥천로 지하162 (판암동),042-539-3601,2022-05-09 00:00:00
1,0102,신흥,S3001,대전 도시철도 1호선,Sinheung,新興,일반역,NaN,NaN,36.319652,127.448792,대전교통공사,대전광역시 동구 옥천로 지하 70 (판암동),042-539-3602,2022-05-09 00:00:00
2,0103,대동,S3001,대전 도시철도 1호선,Dae-dong,大洞,일반역,NaN,NaN,36.329532,127.442848,대전교통공사,대전광역시 동구 계족로 지하 176 (대동),042-539-3603,2022-05-09 00:00:00
3,0104,대전,S3001,대전 도시철도 1호선,Daejeon,大田,일반역,NaN,NaN,36.331583,127.433118,대전교통공사,대전광역시 동구 중앙로 지하 218 (중동),042-539-3604,2022-05-09 00:00:00
4,0105,중앙로,S3001,대전 도시철도 1호선,Jungangno,中央路,일반역,NaN,NaN,36.328664,127.425842,대전교통공사,대전광역시 중구 중앙로 지하 145 (은행동),042-539-3605,2022-05-09 00:00:00


In [4]:
# df.to_csv('./title.csv',index=False)

In [13]:
df = pd.read_csv('./total_info.csv')
df = df[['역사명','역사도로명주소','운영기관명']]
df.rename(columns={'역사명':'지하철역','역사도로명주소':'도로명주소'},inplace=True)
df.운영기관명.unique()
exclude_list = ['대전교통공사', '대구도시철도공사', '부산광역시 부산교통공사', '부산-김해경전철㈜','광주광역시 도시철도공사']
for exclude in exclude_list:
    df = df[df['운영기관명'] != exclude]
df.head()
df.drop(columns=['운영기관명'],inplace=True)
df.shape

(782, 2)

In [7]:
df.to_csv('test_local_info.csv',index=False)

In [16]:
df.reset_index(drop=True,inplace=True)
df.nunique()

지하철역     702
도로명주소    738
dtype: int64

In [17]:
df.head()

,지하철역,도로명주소
0,계양,인천광역시 계양구 다남로 24
1,귤현,인천광역시 계양구 장제로 1136
2,박촌,인천광역시 계양구 장제로 992
3,임학,인천광역시 계양구 장제로 875
4,계산,인천광역시 계양구 경명대로 1089


In [20]:
df['지하철역'][1][:-1]

'귤'

In [21]:
df['지하철역'] = df['지하철역'].str.replace('(', ' ',regex=False,).str.split().str[0]
for i in df.index:
    if df['지하철역'][i][-1] == '역':
        df['지하철역'][i] = df['지하철역'][i][:-1]
df.drop_duplicates(subset=['지하철역'],keep='first',inplace=True)
df.head()

,지하철역,도로명주소
0,계양,인천광역시 계양구 다남로 24
1,귤현,인천광역시 계양구 장제로 1136
2,박촌,인천광역시 계양구 장제로 992
3,임학,인천광역시 계양구 장제로 875
4,계산,인천광역시 계양구 경명대로 1089


In [22]:
df.nunique()

지하철역     656
도로명주소    648
dtype: int64

In [62]:
####################################################################

In [30]:
df.to_csv('test_local_info.csv',index=False)
# 하고 나서 영종 -> 강조문자 삭제
df = pd.read_csv('test_local_info.csv')

In [113]:
# 1
df = pd.read_csv('local_info.csv')

In [114]:
df.head()

,지하철역,도로명주소
0,계양,인천광역시 계양구 다남로 24
1,귤현,인천광역시 계양구 장제로 1136
2,박촌,인천광역시 계양구 장제로 992
3,임학,인천광역시 계양구 장제로 875
4,계산,인천광역시 계양구 경명대로 1089


In [115]:
# 2
def kakao_location(place):
    with open('../03.DataAnalysis/04.지도시각화/data/kakaoapikey.txt') as f_:
        kakao_key = f_.read()
    base_url = "https://dapi.kakao.com/v2/local/search/address.json"
    url = f'{base_url}?query={quote(place)}'
    header = {'Authorization':f'KakaoAK {kakao_key}'}
    result = requests.get(url, headers=header).json()
    lat_ = float(result['documents'][0]['y'])
    lng_ = float(result['documents'][0]['x'])
    return lat_,lng_

In [116]:
# 2
def rtn_addr(target):
    str_addr = df[df.지하철역 == target].도로명주소.values[-1]
    return str_addr.strip()

In [26]:
rtn_addr(df['지하철역'][1])

'인천광역시 계양구 장제로 1136'

In [27]:
kakao_location(rtn_addr(df['지하철역'][1]))

(37.5664130150675, 126.742648844783)

In [117]:
# 3
temp1 =[]
for i in df.index:
    try:
        target = df['지하철역'][i].strip()
        temp1.append(kakao_location(rtn_addr(target)))
    except:
        print(i, df.지하철역[i])
        
df_test = pd.DataFrame(temp1,columns=('lat','lng'))
df2 = pd.concat([df, df_test], axis=1)

In [121]:
# 4
df2.to_csv('stn_r_addr_final.csv',index=False)

In [58]:
temp = df['지하철역'][430]
temp

'성환'

In [59]:
rtn_addr(temp)

'충남 천안시 서북구 성환읍 성롼1로 237-5(성환리 449-128)'

In [112]:
kakao_location('경기도 안산시 단원구 산단로 지하 70')

(37.3019944473115, 126.786725377711)

In [82]:
df3 = df[df['도로명주소'].str.contains('부산|울산')]
df3

,지하철역,도로명주소
555,부전,부산광역시 부산진구 부전동 280-76
556,거제해맞이,부산광역시 연제구 거제동 619-1
557,거제,부산광역시 연제구 거제동 619-1
558,동래,부산광역시 동래구 낙민동 130-4
559,안락,부산광역시 동래구 안락동 201-2
560,부산원동,부산시 동래구 안락동 52-2
561,재송,부산광역시 해운대구 재송동 933-1
562,센텀,부산광역시 해운대구 우동 1215-32
563,벡스코,부산광역시 해운대구 우동 1111-2
564,신해운대,부산광역시 해운대구 좌동 138-4


In [85]:
df[df.지하철역.values == '태화강']

,지하철역,도로명주소
575,태화강,울산광역시 남구 산업로 654


In [86]:
df = df[~df['도로명주소'].str.contains('부산|울산')]

In [87]:
df.shape

(635, 2)

In [88]:
df = pd.read_csv('./test_local_info.csv')

In [89]:
df[df.지하철역.values == '태화강']

,지하철역,도로명주소
577,태화강,울산광역시 남구 산업로 654


In [90]:
df.shape

(658, 2)

In [91]:
df = df[~df['도로명주소'].str.contains('부산|울산')]

In [92]:
df.shape

(637, 2)

In [93]:
df.to_csv('local_info.csv',index=False)

In [120]:
df2.shape

(637, 4)